# **Organización de Datos 75.06 - Trabajo Práctico N°2**
S30. Estimamos el sueldo de un actor como la inversión dividida entre la cantidad de actores. En base a esta estimación, ¿cuál es el actor con el patrimonio más grande?

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from google.colab import drive
drive.mount('/content/drive')

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

movies = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/movies.parquet').rdd
cast = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/cast.parquet').rdd

In [6]:
actoresPorPeli = cast.map(lambda x: (x.id, 1)).reduceByKey(lambda x,y: x+y)

pelisBug = ['1997-08-20', '2012-09-29' ,'2014-01-01']
moviesTupla = movies.filter(lambda x: x.id not in pelisBug) #Eliminamos peliculas con id bug (fechas en vez de id).
moviesTupla = moviesTupla.map(lambda x: (int(x.id), int(x.budget)))

moviesSueldo = moviesTupla.join(actoresPorPeli).map(lambda x: (x[0], x[1][0] / x[1][1])) #Calculamos el sueldo de cada pelicula.

castTupla = cast.map(lambda x: (int(x.id), int(x.cast_id)))

sueldos = moviesSueldo.join(castTupla).map(lambda x: (x[1][1], x[1][0])) #Tuplas tipo (id_actor, sueldo)

maxPatrimonio = sueldos.reduceByKey(lambda x,y: x+y).takeOrdered(1, key = lambda x: -x[1]) #Calculamos patrimonios de los actores y elegimos el mayor.

actorMaxPatrimonio = cast.filter(lambda x: x.cast_id == maxPatrimonio[0][0]).first()

print("El actor de mayor patrimonio es", actorMaxPatrimonio.name, "de ID", actorMaxPatrimonio.cast_id, "y patrimonio de", maxPatrimonio[0][1], "dolares.")

El actor de mayor patrimonio es Samuel L. Jackson de ID 2231 y patrimonio de 185490023.9509397 dolares.
